In [1]:
!pip install PyMySQL

In [26]:
from flask import Flask,render_template,request  ## flask 라이브러리에서 Flask import
import pymysql
import pandas as pd

In [24]:
app = Flask(__name__)

con = pymysql.connect(host='project-db-campus.smhrd.com', user='sc_24K_bigdata25_p3_4', password='smhrd4',
                      port=3312, db='sc_24K_bigdata25_p3_4', charset='utf8', # 한글처리 (charset = 'utf8')
                      autocommit=True # 결과 DB 반영 (Insert or update)
                     )
cur = con.cursor()

# @app.route('/boot/ex/<i nt:seqidx>/<user_id>')
# def hello_world(seqidx,user_id):
#     return render_template('eltag.html')

#@app.route('/boot/ex/<user_id>')
#def hello_world():
#    return render_template('hello2.html')
@app.route('/boot/ex/<id>')
def test_get(id):
    print(id);
    return render_template('index10.html', id=id)


if __name__ == "__main__":
    app.run(host='localhost', port=8097)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit
127.0.0.1 - - [09/Dec/2024 11:46:22] "GET /boot/ex HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:46:25] "GET /boot/ HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:46:28] "GET /boot HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:46:39] "GET /boot/ex/vncj HTTP/1.1" 200 -


vncj


127.0.0.1 - - [09/Dec/2024 11:46:39] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2024 11:46:39] "GET /video_feed HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:46:39] "GET /boot/img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:50:55] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2024 11:51:50] "GET /boot/ex/vncj HTTP/1.1" 200 -


vncj


127.0.0.1 - - [09/Dec/2024 11:51:51] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2024 11:51:51] "GET /video_feed HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:51:51] "GET /boot/img/아동자세.png HTTP/1.1" 404 -


SyntaxError: invalid syntax (2409517804.py, line 1)

In [11]:
from flask import Flask, render_template, Response
import cv2
import mediapipe as mp
import pandas as pd
import datetime
import time
import pymysql

app = Flask(__name__)

con = pymysql.connect(host='project-db-campus.smhrd.com', user='sc_24K_bigdata25_p3_4', password='smhrd4',
                      port=3312, db='sc_24K_bigdata25_p3_4', charset='utf8', # 한글처리 (charset = 'utf8')
                      autocommit=True # 결과 DB 반영 (Insert or update)
                     )
cur = con.cursor()

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 랜드마크 선택
selected_landmarks = [0] + list(range(11, 17)) + list(range(23, 29))

# 컬럼 이름 정의
columns = ['frame'] + [f'landmark{idx}_{axis}' for idx in selected_landmarks for axis in ['x', 'y']] + \
          ['neck_x', 'neck_y', 'hip_x', 'hip_y']

data = []

def generate_frames():
    cap = cv2.VideoCapture(0)  # 카메라 열기
    frame_idx = 0
    start_time = time.time()
    duration = 30  # 30초 동안 실행

    while cap.isOpened() and time.time() - start_time < duration:
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        frame_data = [frame_idx]

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            for idx in selected_landmarks:
                landmark = landmarks[idx]
                frame_data.extend([landmark.x, landmark.y])

            # 목과 엉덩이 좌표 계산
            neck_x = (landmarks[11].x + landmarks[12].x) / 2
            neck_y = (landmarks[11].y + landmarks[12].y) / 2
            hip_x = (landmarks[23].x + landmarks[24].x) / 2
            hip_y = (landmarks[23].y + landmarks[24].y) / 2
            frame_data.extend([neck_x, neck_y, hip_x, hip_y])

        else:
            frame_data.extend([None, None] * len(selected_landmarks))
            frame_data.extend([None, None, None, None])

        data.append(frame_data)

        # 시각화
        if results.pose_landmarks:
            for idx in selected_landmarks:
                x, y = int(landmarks[idx].x * frame.shape[1]), int(landmarks[idx].y * frame.shape[0])
                cv2.circle(frame, (x, y), 5, (255, 0, 0), -1)

        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    cap.release()
    pose.close()

    # CSV 저장
    user_id = 'id123'
    suffix = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    output_csv_path = f'./BPE/training/{user_id}_{suffix}.csv'
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv_path, index=False)
    print(f"Joint 데이터를 {output_csv_path}에 저장했습니다.")


def recent_order(id) :
    # activity_at -> datetime으로 바꿀수잇는지 물어보기
    sql = 'select flow_ord from tmp where id = %s limit 1'
    cur.execute(sql, (id,))
    flow_ord = cur.fetchone()
    if flow_ord is not None and flow_ord[0] is not None:
        print('가장 최근의 flow_ord :', flow_ord[0])
        return flow_ord[0]
    else:
        print('id에 대한 최근 flow_ord가 없습니다. 또는 flow_ord가 NULL입니다.')
        return 1

def select_ps_data(flow_idx, flow_ord) :
    # flow_ord와 flow_idx에 해당하는 ps_idx를 찾는 쿼리
    sql = 'select ps_idx from flow_order where flow_idx = %s and flow_ord = %s'
    cur.execute(sql, (flow_idx, flow_ord))
    ps_idx = cur.fetchone()  # 쿼리 결과에서 첫 번째 값만 가져옵니다.  
    print(ps_idx)
    return ps_idx

def select_order_data(flow_idx, flow_ord) :
    print('select order data')
    sql = 'select flow_order_idx from flow_order where flow_idx = %s and flow_ord = %s'
    cur.execute(sql, (flow_idx, flow_ord))
    flow_order_idx = cur.fetchone()  # 쿼리 결과에서 첫 번째 값만 가져옵니다.
    print(flow_order_idx)
    return flow_order_idx

def select_flow_data(flow_idx) :
    sql = 'select * from flow_info where flow_idx = %s'
    cur.execute(sql, (flow_idx,))
    flow_info = cur.fetchall()
    print(flow_info)
    return flow_info

def select_pose_data(ps_idx) :
    sql = 'select* from pose_info where ps_idx = %s'
    cur.execute(sql, (ps_idx,))
    ps_info = cur.fetchall()
    print(ps_info)
    return ps_info

def insert_user_data(id, flow_order_idx, activity_at, activity_score) :
    print('insert user data')
    sql = 'insert into user_activity (id, flow_order_idx, activity_at, activity_score) values (%s, %s, %s, %s)'
    cur.execute(sql, (id, flow_order_idx, activity_at, activity_score))
    con.commit()

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/boot/programs/<id>/<flow_idx>')
def index(id, flow_idx):
    flow_ord = recent_order(id)
    ps_idx = select_ps_data(flow_idx, flow_ord)
    flow_order_idx = select_order_data(flow_idx, flow_ord)
    
    # activity_at, activity_score 값을 적절하게 정의합니다. 예시로 현재 시간과 랜덤 점수를 사용
    activity_at = datetime.datetime.now().strftime('%Y-%m-%d')  # 활동 시간
    activity_score = 90  # 예시 점수
    
    insert_user_data(id, flow_order_idx[0], activity_at, activity_score)
    
    ps_idx = select_ps_data(flow_idx, flow_ord)
    flow_info = select_flow_data(flow_idx)
    pose_info = select_pose_data(ps_idx)
    return render_template('programs.html', id=id, flow_idx=flow_idx, flow_title = flow_info[0][2], 
                           ps_name = pose_info[0][1],  # '웃타나사나'
                           ps_time = pose_info[0][2],  # 30
                           ps_exp = pose_info[0][3],   # 자세 설명
                           ps_img = pose_info[0][4],   # '1-1.jpg'
                           ps_cal = pose_info[0][5],   # 100
                           ps_link = pose_info[0][6])  # 링크)
      
if __name__ == "__main__":
    app.run(host='localhost', port=8097)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit
[2024-12-11 17:50:19,355] ERROR in app: Exception on /boot/programs/b/4 [GET]
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\smhrd\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\smhrd\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\smhrd\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "C:\Users\smhrd\AppData\Local\Temp\ipykernel_15292\3997853541.py", line 151, in index
    insert_user_data(id, flow_order_idx[0], activity_at, activity_score)
TypeE

가장 최근의 flow_ord : 1
None
select order data
None


127.0.0.1 - - [11/Dec/2024 17:51:00] "GET /boot/programs/b/3 HTTP/1.1" 200 -


가장 최근의 flow_ord : 1
(31,)
select order data
(41,)
insert user data
(31,)
((3, '초급', '체중 감량을 위한 요가 시퀀스', '체중 감량은 점점 더 강박관념이자 정신적 도전이 되고 있습니다. 투쟁은 현실이며 많은 개인이 한계를 넘고, 더 나쁜 것은 자신의 몸을 싫어하기 시작한다는 것입니다! 놀랍지 않게도, 대부분의 사람들은 요가가 스트레칭과 더 관련이 있고 칼로리 소모가 적기 때문에 체중 감량에 도움이 되지 않는다고 생각합니다. 요가는 즉각적인 결과를 제공하지 않을 수 있지만, 근육과 조직의 점진적인 스트레칭 과 강화는 결국 전반적으로 탄력 있는 모습을 제공합니다. 학생들이 천천히 시작하도록 격려하고, 신체와 신체의 한계를 이해하도록 돕습니다. 가장 중요한 것은, 그들에게 인내심을 갖는 법을 가르치는 것입니다. 느리지만 점진적인 방식으로 체중을 감량하는 것이 더 큰 영향을 미치고, 근육에 해를 끼치지 않으며 중요한 미네랄을 고갈시키지 않습니다.', None),)
((31, '사라바사나', 30, '사라바사나(Sarvasana)는 요가에서 "모든 자세" 또는 "전체 자세"라는 의미를 가진 자세로, 일반적으로 "완전한 이완 자세"로 알려져 있습니다. 이 자세는 신체를 완전히 이완시키고, 정신적으로도 완전한 휴식을 취하는 데 중점을 둡니다. 사라바사나는 수련 후에 휴식을 취하는 중요한 자세로, 몸과 마음의 균형을 맞추고 에너지를 재충전하는 데 도움이 됩니다.', '3-2.jpg', 100, None),)
가장 최근의 flow_ord : 1


[2024-12-11 17:51:00,589] ERROR in app: Exception on /boot/programs/b/None [GET]
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\smhrd\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\smhrd\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\smhrd\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "C:\Users\smhrd\AppData\Local\Temp\ipykernel_15292\3997853541.py", line 151, in index
    insert_user_data(id, flow_order_idx[0], activity_at, activity_score)
TypeError: 'NoneType' object is not subscriptable
127.0.0.1

None
select order data
None


127.0.0.1 - - [11/Dec/2024 17:51:00] "GET /boot/programs/b/images/yogapose/10-2.jpg HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:00] "GET /boot/programs/b/images/yogapose/1-1-1.png HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:00] "GET /boot/programs/b/images/yogapose/10-5-1.png HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:00] "GET /boot/programs/b/css/fire.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:00] "GET /boot/programs/b/css/programs.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:00] "GET /boot/programs/b/images/yogapose/10-3.jpg HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:01] "GET /boot/programs/b/js/programs.js HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:01] "GET /boot/programs/b/images/yogapose/10-4.jpg HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:01] "GET /boot/programs/b/images/yogapose/10-6.jpg HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:01] "GET /boot/programs/b/images/yogapose/10-7.jpg HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 

가장 최근의 flow_ord : 1
(13,)
select order data
(48,)
insert user data
(13,)
((5, '초급', '차크라 요가 시퀀스', '차크라 또는 에너지 센터의 균형을 맞추는 것은 개인이 조화와 완벽한 건강을 경험하는 데 필수적입니다. 차크라의 균형이 부족한 요가 학생은 신체적 에너지와 정신적 체력이 낮고 감정적 웰빙이 손상될 수 있습니다. 잘 조율된 아사나와 프라나야마 세트를 연습하면 7개 차크라에서 에너지 효율성을 균형 잡는 데 도움이 될 수 있습니다. 미묘한 에너지의 7개 핵심 포인트 또는 센터는 척추 밑부분에서 머리 꼭대기까지 수직으로 배열되어 있습니다. 차크라를 균형 잡는 것은 인체의 7개 미묘한 에너지 포인트를 제어하는 ??것을 목표로 합니다. 차크라는 신체적, 정신적, 정서적 건강과 직접적인 관련이 있습니다. 각 차크라의 에너지는 신체 기능을 통제하는 데 도움이 됩니다(내부와 외부의 세속적 상호작용 모두). 신체 자체에는 치유하고 균형을 잡을 힘이 있습니다. 차크라에 균형이 부족하면 신체 기능의 불균형이 나타나 전반적인 건강에 부정적인 영향을 미칩니다.', None),)
((13, '단다아사나', 30, '단다아사나(Dandasana), 또는 "지팡이 자세"는 요가에서 기본적인 앉는 자세 중 하나로, 주로 척추를 곧게 펴고, 하체와 코어를 강화하는 데 도움을 줍니다. 이 자세는 요가 수련에서 기초적인 자세로 자주 사용되며, 다른 많은 자세들로 전환하는 시작점이 되기도 합니다. 단다아사나는 전신의 균형을 잡고, 신체의 정렬을 개선하는 데 유익한 자세입니다.', '5-1.jpg', 100, 'https://www.youtube.com/embed/r5Tqo-9mfLM'),)


127.0.0.1 - - [11/Dec/2024 17:51:27] "GET /boot/programs/b/css/programs.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:27] "GET /boot/programs/b/images/yogapose/10-3.jpg HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:27] "GET /boot/programs/b/js/programs.js HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:27] "GET /boot/programs/b/images/yogapose/10-4.jpg HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:27] "GET /boot/programs/b/images/yogapose/10-6.jpg HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:27] "GET /boot/programs/b/images/yogapose/10-7.jpg HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:27] "GET /boot/programs/b/images/yogapose/10-8.jpg HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:27] "GET /boot/programs/b/images/yogapose/10-5-1.png HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:27] "GET /boot/programs/b/images/yogapose/1-1-1.png HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2024 17:51:27] "GET /boot/programs/b/images/yogapose/10-2.jpg HTTP/1.1" 404 -
127.0.0.1 - - [